In [ ]:
# pulls repo to colab
!git clone https://github.com/char-tan/mode_connectivity.git

In [ ]:
# moves file from mode_connectivity/train mode_connectivity
# avoids having to mess around with the import paths
!cp mode_connectivity/train/mlp_mnist_train.py mode_connectivity/

In [ ]:
!python mode_connectivity/mlp_mnist_train.py --lr 0.001 --epochs 100